In [1]:
:opt no-lint

# Applicative Functor

## Outline

1. s

## Why `Applicative` functors?

### Our journey until now

The reason we need Applicative Functors (or Applicatives for short) is due to a limitation of regular `Functors`.

To provide a bit of context lets do a quick recap of our abstraction journey of `map` and `Functor`.

We started with a bunch of recursive functions:

```haskell
lowerString :: [Char] -> [Char]
lowerString []     = []
lowerString (x:xs) = toLower x : lowerString xs

addOne :: Num a => [a] -> [a]
addOne []     = []
addOne (x:xs) = (x + 1) : addOne xs

boolToBit :: [Bool] -> [Char]
boolToBit []     = []
boolToBit (x:xs) = (if x then '1' else '0') : boolToBit xs
```

These functions were usefull, but they were limited to applying a specific funtion to a list of specific types. However, we noticed that they had common traits that we could extract and we got the `map` function:

```haskell
map :: (a -> b) -> [a] -> [b]
map _ []     = []
map f (x:xs) = f x : map f xs


lowerString = map toLower

addOne = map (+1)

boolToBit = map (\x -> if x then '1' else '0')
```

The `map` function is a more powerful version of these functions because it's more general. Now, we can apply any function to a list of values. And because its more general, we can recreate the original functions by passing the concrete function we want to apply to the values.

But then, we faced a problem. `map` only works for lists. But there are plenty of structures with values we want to manipulate. And if we implement their own map-like functions:

```haskell
map :: (a -> b) -> [a] -> [b]
map _ []     = []
map f (x:xs) = f x : map f xs



maybeMap :: (a -> b) -> Maybe a -> Maybe b
maybeMap _ Nothing  = Nothing
maybeMap f (Just x) = Just (f x)



treeMap :: (a -> b) -> Tree a -> Tree b
treeMap f (Leaf x)       = Leaf (f x)
treeMap f (Node lt x rt) = Node (treeMap f lt) (f x) (treeMap f rt)
```

We realize that these functions were useful, but they were limited to specific structures. However, they had common traits that we could extract. And that's how we got the `Functor` type class:

A `Functor` is a type that can apply a function to the values of its structure without modifying the structure itself.

```haskell
class Functor f where
    fmap :: (a -> b) -> f a -> f b
    
    (<$) ::     a    -> f b -> f a
    (<$) =  fmap . const
    {-# MINIMAL fmap #-}
```
**Identity law**
```haskell
fmap id == id
```
**Composition law**
```haskell
fmap (f . g) == fmap f . fmap g
```

The `Functor` type class is a more powerful version of those functions because it's more general. Now, we can apply any function to any structure that is an instance of `Functor`. And, of course, we obtain what we had before, and more, by creating the instances: 

```haskell
instance Functor [] where
  -- fmap :: (a -> b) -> [a] -> [b]
  fmap _ []     = []
  fmap f (x:xs) = f x : fmap f xs


instance Functor Maybe where
  -- fmap :: (a -> b) -> Maybe a -> Maybe b
  fmap _ Nothing  = Nothing
  fmap f (Just x) = Just (f x)
  
  
instance Functor Tree where
  -- fmap :: (a -> b) -> Tree a -> Tree b
  fmap f (Leaf x)       = Leaf (f x)
  fmap f (Node lt x rt) = Node (fmap f lt) (f x) (fmap f rt)
```

As you can see, every time we extracted a more general expression we got a more powerful abstraction. But there are cases when `Functor` is not enough.

### The limits of `Functor`

Let's try applying a few functions to a `Functor`. Let's say the `Maybe` functor: 

In [32]:
:t (+1)             -- Int -> Int
:t (+1) <$> Just 3  -- Maybe Int
(+1) <$> Just 3

-----------------------------------------------------------------

-- Add (Just 3) and (Just 2)?
:t (+)             -- Int -> Int -> Int
:t (+) <$> Just 3  -- Maybe (Int -> Int)

-----------------------------------------------------------------

almostThere = (<$> Just 3) <$> ((+) <$> Just 2) 

:t  almostThere -- Maybe (Maybe Int)
almostThere

(+1) :: forall a. Num a => a -> a

(+1) <$> Just 3 :: forall b. Num b => Maybe b

Just 4

(+) :: forall a. Num a => a -> a -> a

(+) <$> Just 3 :: forall a. Num a => Maybe (a -> a)

almostThere :: forall b. Num b => Maybe (Maybe b)

Just (Just 5)

As you can see with the first example, we have no issue fmapping the a function +1 to Just 3. But what if we want to add two functors? If we check the result of applying the plus operator to Just 3, we get a function at the functor level. We are not prepared for this!

We can use fmap to take regular functions and apply them to functors. But we don't have a way to apply functions that already are at the functor level to functors. In the last example, we did some juggling to get almost what we needed. We managed to add the values but we ended up with duplicated functors.

The worst thing is that, this is a really common scenario. We may want to add two optional values that we retrieved from our database, maybe we want to apply a function to combine 3 trees into one, it's more common to have to work with multiple functors than with a single one. So, we need to solve this. 


One solution would be to create another versions of `fmap` that can handle a function of two arguments. For example, like this:

In [35]:
fmap2 :: (a -> b -> c) -> Maybe a -> Maybe b -> Maybe c
fmap2 f (Just a) (Just b) = Just (f a b)
fmap2 _ _ _               = Nothing



fmap2 (+) (Just 3) (Just 2)

Just 5

We take a binary function and two `Maybe` functors. If both values are `Just`, we apply the binary function, else, we return nothing because, if we are missing one or both values, we have no way of returning a valid result.

That would solve the problem. So in theory, we could create a type class called, for example:

```haskell
class Functor2 where
    fmap2 :: (a -> b -> c) -> f a -> f b -> f c
```

But that only works for binary functions. If we need to use a function that takes three, four, or five arguments, we would need to create:

```haskell
class Functor3 where
    fmap3 :: (a -> b -> c -> d) -> f a -> f b -> f c -> f d

class Functor4 where
    fmap4 :: (a -> b -> c -> d -> e) -> f a -> f b -> f c -> f d -> f e

class Functor5 where
    fmap5 :: (a -> b -> c -> d -> e -> g) -> f a -> f b -> f c -> f d -> f e -> f g
```

You know where we're going with this. This method is unsustainable. It doesn't matter how many type classes you create, there's always a usecase that requires more parameters. And, on top of that, we would have maybe dozens of type classes representing the same concept. 

But don't worry, thanks to currying, there's a better way!

# Function application at the `Functor` level 🥇

This is the key idea of the lesson. If yo understand this section, the rest of the lecture will flow naturally.

Ok, so, we know that writing an `fmap` for every possible number of arguments is not feasable. But we don't have to. If we take a look at Haskell's built in function application:

```haskell
( ) :: (a -> b) -> a -> b

result = (+1) 3 -- 4
--           ^
--           |
--    Function application
```

It takes a function that goes from `a` to `b` and a value of type `a` and returns a value of type `b`.

And, because in Haskell, all functions are curried, meaning they all take one argument, we can apply this operator multiple times until our funciton is fully applied:

In [36]:
calculate :: Int -> Int -> Int -> Int
calculate a b c = a + b * c

calculate 3 6 4
--       ^ ^ ^
--       | | |
--    Function application

27

That's what we need to achieve. An operator that works like function application, but for functors. That way, it will work with functiuons of any number of arguments.

So, if regular function application has this signature:

```haskell
( ) ::                 (a -> b) -> a -> b
```

How does the type signature of the same function would look if everything was a `Functor`? Well, you guessed it, like this:

```haskell
(<*>) :: Functor f => f (a -> b) -> f a -> f b
```

We call this the "star" or "ap" operator. We use an operator because, of course, it will be used mostly as an infix function. And the type reflects that the behavior is the same as regular function application but lifted to work at the functor level.

It's important not to confuse the signatures of this functor-level function application with fmap:

```haskell
(<$>) ::                (a -> b) -> f a -> f b
```

`fmap` takes a function that is not at the functor level and applies it to a functor. Once you applied the function the first time, the end result, be it the final value or a partially applied function, is now at the functor level. So we can not use `fmap` anymore.

Ok, so let's try defining it specifically for optional types. For `Maybe` values, this star operator would look like this: 

```haskell
(<*>) :: Maybe (a -> b) -> Maybe a -> Maybe b
Nothing <*> _       = Nothing     -- We don't have function 🤷
_       <*> Nothing = Nothing     -- We don't have a value to apply the function to 🤷
Just f  <*> Just x  = Just (f x) 
```

We have to patternmatch to make sure that we actually have both the function and the value. If we don't have both, we can't return a valid result, so we return `Nothing`. Else, we patternmatch to extract the function and value, apply the funciton to get a value of type `b`, and wrap it again with the Just constructor.

This is really similar to the `fmap` definition. With the caveat that now we have to take into account that the function is optional too, not just the value.

Now that we have our definition, let's try it out!:

In [40]:
Just (+1) <*> (Just 3)

Just (+) <*> (Just 3) <*> (Just 2)

calculate :: Int -> Int -> Int -> Int
calculate a b c = a + b * c

Just calculate <*> (Just 3) <*> (Just 2) <*> (Just 4)

Just 4

Just 5

Just 11

Awesome, it seems that it works. But, I asured you that the star operator is function application at the functor level, and I compared its type with the function application type to build my case. It's time I prove it to you.

To prove that the star operator is function application at the functor level, I'm going to create a type that doesn't do anything. If the structure doesn't do anything, then the star operator should return the same result as regular function application, right? So let's do that.

How do you imagine a type that doesn't do anything looks like? It's actually pretty straight forward. We need a type that wraps another type and doesn't provide any functionality. Like this:

In [41]:
newtype Identity a = Identity { getIdentity :: a } deriving (Show, Eq)

As you can see, if we have a type with only one constructor that the only thing it does is to hold the value inside. It's virtually the same as the underlying type but with the extra anoyance of having to wrap and unwrap it.

We call this type the `Identity` type for the same reasons we call the identity values and the identity function like that. They preserve the identity of the values they interact with. They don't do anything basically.

Remember the newtype wrappers of the Semigroup and Monoid lessons? Those weren't identities because the behaviour we gave them when defining their Semigroup and Monoid instances actually did something. So now, we have to also implement the type class instances in a way that they have no effect.

Ok, so, let's implement the `Functor` instance for the `Identity` type:

In [42]:
instance Functor Identity where
  --fmap :: (a -> b) -> Identity a -> Identity b
  fmap f (Identity a) = Identity (f a)

To do that, we'll just pattern match to extract the internal value, apply the function and wrap it again. It's the same as the `Just` case of the `Maybe` type, with the difference that, because we only have one constructor, there's no option to fail, so we'll always be able to apply the function to the underlying value.

In [43]:
fmap (+1) (Identity 3) -- Identity 4

(*2) <$> Identity 3    -- Identity 6

'A' <$ Identity 3      -- Identity 'A'

fmap id (Identity False) == Identity False -- True

Identity {getIdentity = 4}

Identity {getIdentity = 6}

Identity {getIdentity = 'A'}

True

And as you can see, it works as expected and respects the `Functor` laws.

Now, we can create the star operator. 

The type was:

```haskell
(<*>) :: Functor f => f (a -> b) -> f a -> f b
```

So, we had a funciton at the functor level applied to a functor with values of type `a` to get the functor with values of type `b`. Because the `Identity` functor is basically just an anoying wrapper, for now at least, we know that we just have to unwrap and apply the funciton to the value of type `a`. So let's do that:

In [45]:
(<*>) :: Identity (a -> b) -> Identity a -> Identity b
(Identity f) <*> (Identity x) = Identity (f x)

infixl 4 <*>

Pretty straight forward, almost the same as `fmap`. Now comes the moment of truth. Let's see if the star operator is actually function application at the functor level:

In [46]:
(+1) 1
getIdentity $ Identity (+1) <*> Identity 1

(+) 1 2
getIdentity $ Identity (+) <*> Identity 1 <*> Identity 2

(\a b c -> a + b * c) 1 2 3
getIdentity $ Identity (\a b c -> a + b * c) <*> Identity 1 <*> Identity 2 <*> Identity 3

2

2

3

3

7

7

As you can see, we have three examples of applying the same funtion to a value or multiple values. Once without functor using the built-in function application, and once using the star operator within a `Functor` that doesn't do anything. And we get the same result in both cases.

We just proved that the star operator is a more generalized form of function application. It's more general because we get the same old function application if we apply it to the `Identity` functor, but we can also apply it to other functors.

--- Alternative: Is this enough though? What if one of the values is not a functor? For example, what if we want to combine three values, one local, and two that come from a database so they are maybe values. Like this:

--- Alternative
```haskell
--                              DB      Local    DB
Just (\a b c -> a + b * c) <*> Just 1 <*> 2 <*> Just 3
--                                        ^
--                                        |
--                              ❌  Not a Functor! ❌
```

As of now, we don't have a way to lift a value. We can only lift a function using fmap, and all other values have to already be functors.

Now that we have our new operator, let's see if it actually solves the problem we had.

## Being `pure` 😇

We proved that the star operator is a generalized form of function application.  we could use it to build `fmap`s for every function. So let's try to do that:

```haskell
    fmap0 :: a -> f a
    
    fmap1 :: (a -> b) -> f a -> f b
    fmap1 = fmap

    fmap2 :: (a -> b -> c) -> f a -> f b -> f c

    fmap3 :: (a -> b -> c -> d) -> f a -> f b -> f c -> f d

    fmap4 :: (a -> b -> c -> d -> e) -> f a -> f b -> f c -> f d -> f e

    fmap5 :: (a -> b -> c -> d -> e -> g) -> f a -> f b -> f c -> f d -> f e -> f g
```

I'm not adding the `Functor` constraint for readibiliy, but all those `f`s are functors.

Ok, so lets try that. We already have `fmap1` (the `fmap` that lifts a function of one argument). It's our regular old `fmap`. Bput we'll redefine it using our new operator.

Let's start with the simplest of all: `fmap0`. It takes a function that takes zero inputs, so, a constant value, and returns the same value lifted to the functor level. Hmm... righ of the bat we have an issue here. How do we do this? I mean, I don't see how we could use the `fmap` or the star operator. `fmap` requires two arguments and the star operator as well, on top of requiring that everything has to already be at the functor level. 

What about specific function, though? Could we think of ways to implement `fmap0` for specific functors? I think so. And it's acually pretty easy!

For example, how can we embed a value of type `a` inside the `Identity` functro? Easy:

In [79]:
fmap0 :: a -> Identity a
fmap0 = Identity


fmap0 3
fmap0 "Hellooo!"
fmap0 False

Identity {getValue = 3}

Identity {getValue = "Hellooo!"}

Identity {getValue = False}

We just wrapp the value with the `Identity` constructor.

And, if we want to do it for `Maybe` values, it would be like this:

In [80]:
fmap0 :: a -> Maybe a
fmap0 = Just


fmap0 3
fmap0 "Hellooo!"
fmap0 False

Just 3

Just "Hellooo!"

Just False

We wrapp with the `Just` constructor. We pick `Just` instead of `Nothing` because else we'd always get `Nothing` and we'd have a function that alway fails. But we're not failing, we already know the value, so we can just embed it inside the `Just`.

We can do the same for lists:

In [82]:
fmap0 :: a -> [a]
fmap0 x = [x]


fmap0 3
fmap0 "Hellooo!"
fmap0 False

[3]

["Hellooo!"]

[False]

A list of a single value is still a list.

Etcetera, etcetera.

So, we can write this function for individual functors, but we have to do it more generally, for every functor. And because every functor works in different ways, we'll have to define a new function that does this for us as part of a type class. We'll call this fucntion `pure`:

```haskell
pure :: a -> f a
```

`pure` takes a value and embedes it in a functor. That's it. We call it "pure" because...

## Defining every `fmapX` following the types

Ok, so, now that we have pure, let's define all those `fmapN` functions. Just to prove that we don't need them if we ouse these two new functions. Starting, again, with `fmap0`:

```haskell
fmap0 :: a -> f a
fmap0 = ..
```

We need to take a constant value and lift it to the functor level. Easy, we just defined `pure` to do exactly that, so our first `fmap` is:

```haskell
fmap0 :: a -> f a
fmap0 = pure
```

One down, 5 to go. What about `fmap1`?

```haskell
fmap1 :: (a -> b) -> f a -> f b
fmap1 = fmap
```

I know it's the same as `fmap`, but we said we could define everything uisng just:

```haskell
pure a -> f a
(<*>) f (a -> b) -> f a -> f b
```

So, how are we gonna do this? Let's start from scratch:

```haskell
fmap1 :: (a -> b) -> f a -> f b
fmap1 g fx = ...
```

```haskell
g  :: a -> b
fx :: f a
pure  :: a -> f a
(<*>) :: f (a -> b) -> f a -> f b
---------------------------------------------------------------------------------------------
pure g          :: f (a -> b)                -- lift g
---------------------------------------------------------------------------------------------
(<*>)           :: f (a -> b) -> f a -> f b  -- We can use <*> directly
pure g <*>      ::               f a -> f b  -- partially apply <*>
---------------------------------------------------------------------------------------------
pure g <*> fx   ::                      f b  -- fully apply <*>
```

```haskell
fmap1 :: (a -> b) -> f a -> f b
fmap1 g x = pure g <*> x -- = fmap g x = g <$> x
```

And that's how we define `fmap1` which is the same as `fmap`. Now, we can do `fmap2`:

```haskell
fmap2 :: (a -> b -> c) -> f a -> f b -> f c
fmap2 g fx fy = ...
```

So, we have the function `g` that takes a value of type `a` and another of type `b`, one value of type `f a` and another of type `f b`. And, using those and the two funcitons we just defined, we want to obtain `f c`.

```haskell
g  :: a -> b -> c
fx :: f a
fy :: f b
pure  :: a -> f a
(<*>) :: f (a -> b) -> f a -> f b
---------------------------------------------------------------------------------------------
pure g               :: f (a -> b -> c)  -- lift g
---------------------------------------------------------------------------------------------
(<*>)                :: f (a -> (b -> c)) -> f a -> f (b -> c)  -- specialize `b` to `b -> c`
pure g <*>           ::                      f a -> f (b -> c)  -- partially apply <*>
---------------------------------------------------------------------------------------------
(pure g) <*> fx      ::                             f (b -> c)  -- fully apply <*>
---------------------------------------------------------------------------------------------
(<*>)                ::                             f (b -> c) -> f b -> f c -- spec `a` to `b`, `b` to `c`
(pure g) <*> fx <*>  ::                                           f b -> f c -- partially apply second <*>
---------------------------------------------------------------------------------------------
pure g <*> fx <*> fy ::                                                  f c -- fully apply second <*>
```

```haskell
fmap2 :: (a -> b -> c) -> f a -> f b -> f c
fmap2 g fx fy = pure g <*> fx <*> fy
```

We can keep going, but for the sake of time, I'll asume you figure out the pattern. Once we lift the function using pure, all we have to do is use the star operator as function application for the rest of the parameters and we're good. So, this is how all the fmap functions would look like: 

```haskell
    fmap0 :: a -> f a
    fmap0 = pure
    
    fmap1 :: (a -> b) -> f a -> f b
    fmap1 g fa = pure g <*> fa -- same as g <$> fa

    fmap2 :: (a -> b -> c) -> f a -> f b -> f c
    fmap2 g fa fb = pure g <*> fa <*> fb

    fmap3 :: (a -> b -> c -> d) -> f a -> f b -> f c -> f d
    fmap3 g fa fb fc = pure g <*> fa <*> fb <*> fc

    fmap4 :: (a -> b -> c -> d -> e) -> f a -> f b -> f c -> f d -> f e
    fmap4 g fa fb fc fd = pure g <*> fa <*> fb <*> fc <*> fd

    fmap5 :: (a -> b -> c -> d -> e -> g) -> f a -> f b -> f c -> f d -> f e -> f g
    fmap5 g fa fb fc fd fe = pure g <*> fa <*> fb <*> fc <*> fd <*> fe
```

On top of that, we see that from fmap1 onwards, we can replace the two steps of fist lifting the function with `pure` and then applying it with the star operator with one step using the fmap operator. So the same expresions could be written like this:

```haskell
    fmap0 :: a -> f a
    fmap0 = pure
    
    fmap1 :: (a -> b) -> f a -> f b
    fmap1 g fa = g <$> fa

    fmap2 :: (a -> b -> c) -> f a -> f b -> f c
    fmap2 g fa fb = g <$> fa <*> fb

    fmap3 :: (a -> b -> c -> d) -> f a -> f b -> f c -> f d
    fmap3 g fa fb fc = g <$> fa <*> fb <*> fc

    fmap4 :: (a -> b -> c -> d -> e) -> f a -> f b -> f c -> f d -> f e
    fmap4 g fa fb fc fd = g <$> fa <*> fb <*> fc <*> fd

    fmap5 :: (a -> b -> c -> d -> e -> g) -> f a -> f b -> f c -> f d -> f e -> f g
    fmap5 g fa fb fc fd fe = g <$> fa <*> fb <*> fc <*> fd <*> fe
```

This style of using fmap to start the function aplication and continuing with the star operator is called applicative style, and is really common in Haskell.

Remenber that defining all these fmaps was just to demostrate the usefulnes and power of the pure and star operators. In reality we use the operators directly as we'll do in the rest of the lesson.

OK, that's it! We solve the limitation we suffered with `fmap`. Now, let's make it official by creating the type class.

## The `Applicative` type class

Here's the `Applicative` type class: 

```haskell
class Functor f => Applicative f where
    pure  :: a -> f a
    (<*>) :: f (a -> b) -> f a -> f b
```

So, how can we express what an `Applicative` is in words? For starterd, could say something like this:

An `Applicative` is a functor with a function to embed expressions (`pure`), and an operator that works as function application at the functor level (`<*>`).

That's why they are called applicative functors, because they are functors with function application.

To define the `Applicative` type class, we just need these two new behaviours we just discovered. But before we formally define it, let's make it more concrete by defining a few instances so we have 

Let's start by defining the instance for the `Identity` type wrapper:

In [13]:
newtype Identity a = Identity { getValue :: a } deriving (Show, Eq)

instance Functor Identity where
  --fmap :: (a -> b) -> Identity a -> Identity b
  fmap f (Identity a) = Identity (f a)

Of course, because `Applicative` is a subclass of `Functor`, we have to define `Functor` first. It's pretty straight forward. We pattern match to extract the value wraped with the `Identity` constructor, apply the function to the underlying value, and wrap it again. Now to the new part:

In [19]:
instance Applicative Identity where
  -- pure :: a -> Identity a
  pure = Identity
  -- (<*>) :: Identity (a -> b) -> Identity a -> Identity b
  -- Identity f <*> Identity a = Identity (f a)
  Identity f <*> x = f <$> x -- same as definition above

To create an instance for the applicative type class, we need to define both `pure` and the star operator. We don't have much choice for `pure`. We need to lift a value to the `Identity` level and we have only one constructor with the same type as pure. So we just use that.

For the star operator, we have two choices. We can patternmatch to extract both the function and the value, apply the function to the value and wrap it again, or because we have `fmap` at our disposal, we can patternmatch to extract only the funciton and fmappit to the value. Which, if you look at the definition of the functior instance, it's the same as what we did in the first definition by hand, but we avoid repeating ourselves.

Now that the `Identity` type is an applicative functor, we can use the operators:

In [21]:
Identity (+1) <*> Identity 1

Identity (+) <*> Identity 1 <*> Identity 2

Identity (\a b c -> a + b * c) <*> Identity 1 <*> Identity 2 <*> Identity 3

Identity {getValue = 2}

Identity {getValue = 3}

Identity {getValue = 7}

That one was borring, so let's do a new one. Let's create an instance for the `Maybe` type. Same as before we start with the functor instance:

```haskell
data Maybe a = Nothing | Just a

-------------------------------------------------------------------------

instance Functor Maybe where
  -- fmap :: (a -> b) -> Maybe a -> Maybe b
  fmap _ Nothing  = Nothing
  fmap f (Just x) = Just (f x)

-------------------------------------------------------------------------

instance Applicative Maybe where
  -- pure :: a -> Maybe a\
  -- pure = \x -> Nothing ❌
  pure = Just
  -- (<*>) :: Maybe (a -> b) -> Maybe a -> Maybe b
  Nothing <*> _     = Nothing
  _ <*> Nothing     = Nothing
  Just f <*> Just x = Just (f x)
```

We don't run this cell because these instances are alreay provided by the prelude.

So, we alredy talked about the `Functor` instance in the previous lesson, so we won't go there. Now, about the applicative instance...

We have two possible options that satisfy the types for `pure`. We could ignore the input and return `Nothing`, and we could use `Just` that fits perfectly fine by itself. Conceptually, it makes sense to .... . But, if that doesn't convince you, `Just` is the only options if we want to respect the laws of the type class!

## The `Applicative` laws

Remember that one of the characteristics of functors was that `fmap` din't change the structure itself. Here, the star operator has the same restriction.

Sadly, same as it happened with Semigroup, Monoid, and Functor, not everything is reflected in the type class definition. So, we have to complement it with a few laws. Here they are:

**Identity:**
```haskell
pure id <*> v = v  -- pure id <*> Just 3 = Just (id 3) = Just 3
```

**Homomorphism:**
```haskell
pure f <*> pure x = pure (f x) -- pure show <*> pure 3 = pure (show 3)
```

**Composition:**
```haskell
(pure (.) <*> u <*> v) <*> w = u <*> (v <*> w) 
```

This one shows that the operator is associative, although it might not be clear at a glance, so here's an example:

In [5]:
left  = (pure (.) <*> Just show <*> Just (*2)) <*> Just 3
right =               Just show <*> (Just (*2) <*> Just 3)

left
right
left == right

Just "6"

Just "6"

True

On the left side, we want to combine the `show` function with the  `(*2)` funtion first and then apply it to `3`. And we know that the way of combining functions is by composition. So, we lift function composition to the functor level and apply it to `Just show` and `Just (*2)`. Then, we apply the result of that comoposition to `Just 3`.

On the right side, we fist want to combine the last two and then combine the result with the first. In this case, we don't need to lift function composition because we can apply `Just (*2)` to `Just 3` directly. So we do that to get `Just 6` and then apply `Just show` to get the string representation of the number six.

As you can see, we get the same result on both sises.

Finally, we have the interchange law:

**Interchange:**
```haskell
f <*> pure x = pure (\f -> f x) <*> f
```

This one states that, we should get the same result if we flip the arguments when applying a function. We know this is true for regular function application. For example:

In [9]:
left  = (+1) 3
right = (\f -> f 3) (+1) 

left
right
left == right

4

4

True

On the left side, we take the function `(+1)` and apply it to 3 to get 4.

On the right side, we take a function that takes another function as parameter and applies it to the number three and we apply this function to the `(+1)` funciton. This results in +1 being passed as parameter and being applied to 3.

As you can see, we have to make the extra step of adding a lambda function, but if we flip the arguments of the space function application operator, we stil get the same result. That property should also hold with our star operator:

In [8]:
left  = Just (+1) <*> pure 3
right = pure (\f -> f 3) <*> Just (+1) 

left
right
left == right

Just 4

Just 4

True

This is the same as we just saw but lifted at the functor level. Well, now that we have the `Applicative` type class, lifted at the `Applicative` level.

Same as before, we have to make the extra step of adding a lambda function, but if we flip the arguments of the star operator, we stil get the same result.

## 🎆 Programming with effects 🎆

Congratulations! You now know how to program with effects!! Oh, you didn't realize that? Well, you do! Here's why:

An `Applicative` is a functor with operations to embed pure expressions (`pure`), and sequence computations and combine their results (`<*>`).

---

---

---

# Applicative

## Outline

* Incentive for Applicative

* Definition of Applicative type class

* Using Applicative and Functor together

* Applicative examples

* Applicative laws

In this lesson, we will learn about the Applicative type classe and how you can use it.

## Incentive for Applicative

Imagine you have a function `maybeAdd1 :: Num a => Maybe (a -> a)`. This means that the function migh exist or not. 

How can this be? Well a practical example is if you use the `<$>` operator with a function like `(+)` and apply it to a `Maybe a` value.

In [ ]:
maybeAddition :: Num a => Maybe a -> Maybe (a -> a)
maybeAddition maybeVal = (+) <$> maybeVal

-- Here is a more detailed breakdown of this function without using <$>
maybeAddition' :: Num a => Maybe a -> Maybe (a -> a)
maybeAddition' Nothing = Nothing
maybeAddition' (Just n) = Just $ (+) n

var1 :: Maybe Int
var1 = Just 1

maybeAdd1 = maybeAddition var1
:t maybeAdd1

We see that the command `:t` in the code above returns the signature we defined in the begining where type `a` is now `Int`. 

Now comes the question if you want to use this function on another `Maybe Int` variable how would you do it? 

The function would then have the type signature:
```haskell
add1 :: Maybe Int -> Maybe Int
```
One possibility would be to write the `add1` function such that handles the case of the missing function.

In [ ]:
import Data.Maybe ( fromJust )

var2 :: Maybe Int
var2 = Just 2

add1 :: Maybe Int -> Maybe Int
add1 Nothing = Nothing
add1 (Just n) = Just $ fromJust maybeAdd1 n

print $ add1 var2

Instead of using pattern matching when defining the `add1` function we can also use the Applicative type class to solve this problem.

## Definition of Applicative type class

The Applicative type class is defined as follows:
```haskell
class Functor f => Applicative f where
  pure :: a -> f a
  (<*>) :: f (a -> b) -> f a -> f b
  GHC.Base.liftA2 :: (a -> b -> c) -> f a -> f b -> f c
  (*>) :: f a -> f b -> f b
  (<*) :: f a -> f b -> f a
  {-# MINIMAL pure, ((<*>) | liftA2) #-}
```

We see that Functor is a superclass of Applicative. This means that every type that has an instance of Applicative also has to have an instance of Functor.

The minimal complete definition requires the `pure` operator and another operator which can be the operator `<*>` called app or the function `liftA2`. 

The `pure` function takes a type and puts it into a context. Here is an example with Maybe and lists which both have an instance of Applicative.

var1 = pure 1 :: Maybe Int
var2 = pure 1 :: [Int]

print var1
print var2

Ussually when we work with a type that has an instance of Applicative we are using the `<*>` operator and more rarely the `liftA2` function.

The `<*>` operator takes a function and a variable both in the same context and applies the function to the variable after taking them out of their context. 

Then it puts the result back in the context. Let's see how it can help us to easier defining the `add1` function.

In [ ]:
var1 = 1 :: Int
var2 = Just 1 :: Maybe Int

add1 :: Maybe Int -> Maybe Int
add1 maybeInt = maybeAdd1 <*> maybeInt

print $ add1 (pure var1)
print $ add1 var2

We get the same result as before and the function is one line shorter. A good reason to use the `<*>` operator is that it's easier to read the code because it's generic. 

The same symbol serves the same purpose for all Applicatives. And for types that have already an instance of Applicative as Maybe and lists, the `<*>` operator is already defined.

## Using Applicative and Functor together

If we think a bit about the examples from the previous chapter we may ask ourselves how to write a function `add` that takes in two Maybe values and returns also a Maybe value.

In [ ]:
add :: Num a => Maybe a -> Maybe a -> Maybe a
add maybeVal1 maybeVal2 = (+) <$> maybeVal1 <*> maybeVal2

print $ add var1 var2

We see we get again the same result and used much less lines of code. Because of how `<*>` is defined you can always add one more parameter to the function. 

Also instead of using the `<$>` operator you can use `pure` that puts the function in the begining directly in the maybe context.

In [ ]:
var3 :: Maybe Int
var3 = Just 3

addThree :: Num a => Maybe a -> Maybe a -> Maybe a -> Maybe a
addThree maybeVal1 maybeVal2 maybeVal3 = pure (+) <*> (pure (+) <*> maybeVal1 <*> maybeVal2) <*> maybeVal3

print $ addThree var1 var2 var3

## Applicative examples

Another way how to use the Applicative type class is in IO which has also an instance of Applicative. 

Let's imagine you ask the user for two numbers and then calculate their product. Here is the code where we make use of the `<*>` and `<$>` operators.

In [ ]:
myProduct :: Num a => a -> a -> a
myProduct x1 x2 = x1*x2

readDouble :: IO Double
readDouble = read <$> getLine

ioProduct :: IO Double
ioProduct = myProduct <$> readDouble <*> readDouble

main :: IO ()
main = do
    print "Input 2 numbers. Use dot for decimal seperator."
    result <- ioProduct
    print result

main

You can also use the `<*>` operator on lists to get all possible combinations. 

This is because if you view a list type as a context instead as a container, you can say that a list represents more possible values for a fixed type. 

For instance `[Int]` represent more possible values of type `Int`. Here is a code example:

In [ ]:
list1 = [1,2,3] :: [Int]
list2 = [4,5] :: [Int]

allCombinations :: Num a => [a] -> [a] -> [a]
allCombinations l1 l2 = pure (+) <*> l1 <*> l2

print $ allCombinations list1 list2

Another example for creating all possible combination is by using lists to create an instance of a user defined type with record syntax. Here is the code:

In [ ]:
data Car = Car {
    company :: String,
    color :: String
} deriving Show

companies = ["Toyota", "Mercedes", "Ford"]
colors = ["yellow", "gree", "blue"]

allPossibleCars :: [Car]
allPossibleCars = pure Car <*> companies <*> colors

print allPossibleCars

In our final example let's create an Applicative instance for the Wrapper type we defined in the previous lesson.
```haskell
data Wrapper a = Empty | Wrapper a deriving Show
```

We will first need to define the function that will work as the `<*>` operator. The pure function we can define directly.

In [ ]:
data Wrapper a = Empty | Wrapper a deriving Show

appWrapper :: Wrapper (a -> b) -> Wrapper a -> Wrapper b
appWrapper f Empty = Empty
appWrapper Empty x = Empty
appWrapper (Wrapper f) (Wrapper n) = Wrapper (f n)

instance Functor Wrapper where
    fmap f val = (pure f) <*> val   

instance Applicative Wrapper where
    (<*>) = appWrapper
    pure val = Wrapper val

Now we can define an IO function that askes the user for two numbers, creates Wraper types from them and summs those numbers together.

In [ ]:
import Data.Char (isDigit)

sumWrapperNums :: IO ()
sumWrapperNums = do
    putStrLn "Input first number:"
    n1 <- getLine
    putStrLn "Input second number:"
    n2 <- getLine

    let test = all isDigit n1 && all isDigit n2
    if test 
    then do
        let w1 = pure (read n1) :: Wrapper Int
            w2 = pure (read n2) :: Wrapper Int
        putStrLn "Wrapper sum is:"
        print $ (+) <$> w1 <*> w2
    else do
        putStrLn "The input data should be only digits. Try again."
        sumWrapperNums

sumWrapperNums

## A more complex example

If we check in Haskell what type classes a function in general belongs to, we see it has also an instance of Applicative.
```haskell
:i (->)
type (->) :: * -> * -> *
data (->) a b
infixr -1 ->
instance Applicative ((->) r) 
instance Functor ((->) r) 
instance Monad ((->) r) 
instance Monoid b => Monoid (a -> b) 
instance Semigroup b => Semigroup (a -> b) 
```

The definition of the Applicative instance for a funtion is as follows:
```haskell
instance Applicative ((->) r) where
    pure = const
    (<*>) f g x = f x (g x)
```

The second definition is valid only if `f` is a function that takes in two parameters and `g` is a function that takes in one parameter. 

Here is an example how we can use this to construct a fibonacci list with `<*>` where we also use a recursive call to the list itself.

In [ ]:
fibs = 0 : 1 : (zipWith (+) <*> tail) fibs

print $ take 5 fibs

## Applicative laws

Same as for Functor the Applicative type class has also its origins in mathematics and is defined with laws.

Applicative has 4 laws. They are as follows:

- Identity:<br>`pure id <*> x = x`

- Composition:<br>`pure (.) <*> x1 <*> x2 <*> x3 = x1 <*> (x2 <*> x3)`

- Homomorphism:<br>`pure f <*> pure x = pure (f x)`

- Interchange:<br>`x1 <*> pure x2 = pure ($ x2) <*> x1`

The identity law ensures that `<*>` should only be a mapping function and should not change anything in the applicative except for the value that it’s mapping.

The composition law says that independent of which way we chose to apply our functions, we should always get the same result.

The homomorphism law says that `<*>` should not be doing anything else then applying the mapping.

The interchange law states that we can flip the operands of `<*>` in a predictable way.

Below is example code that proves these laws hold for the Maybe type which has an instance of Applicative.

In [ ]:
-- Identity law
(pure id <*> (Just 1)) == Just 1
(pure id <*> Nothing) == Nothing

-- Composition law
add1, mult2 :: Maybe (Integer -> Integer)
add1 = (+) <$> (Just 1)
mult2 = (*) <$> (Just 2)

(pure (.) <*> add1 <*> mult2 <*> (Just 1)) == (add1 <*> (mult2 <*> (Just 1)))
(pure (.) <*> Nothing <*> mult2 <*> (Just 1)) == (Nothing <*> (mult2 <*> (Just 1)))

-- Homomorphism law
add2 :: Num a => a -> a
add2 x = x + 2
(pure add2 <*> pure 1 :: Maybe Int) == (pure (add2 1) :: Maybe Int)

-- Interchange law
(add1 <*> pure 1) == (pure ($ 1) <*> add1)

Let's look now at an example where we create a type and an Applicative instance for it that violates these laws.

In [ ]:
data NotOk a = NotOk a Bool deriving (Eq, Show)

appNotOk :: NotOk (a -> b) -> NotOk a -> NotOk b
appNotOk (NotOk f myBool1) (NotOk x myBool2) =
    NotOk (f x) (not myBool2)

pureNotOk :: a -> NotOk a 
pureNotOk x = NotOk x True

instance Functor NotOk where
    fmap f x = pure f <*> x

instance Applicative NotOk where
    pure = pureNotOk
    (<*>) = appNotOk

var1 :: NotOk Int
var1 = NotOk 1 True

add1, mult2 :: NotOk (Int -> Int)
add1 = (+) <$> (NotOk 1 True)
mult2 = (*) <$> (NotOk 2 True)

add2 :: Num a => a -> a
add2 x = x + 2

print $ (pure id <*> var1) == var1
print $ (pure (.) <*> add1 <*> mult2 <*> var1) == (add1 <*> (mult2 <*> var1))
print $ (pure add2 <*> pure 1 :: NotOk Int) == (pure (add2 1) :: NotOk Int)
print $ (add1 <*> pure 1) == (pure ($ 1) <*> add1)

Same as for Functor it is not obligatory to follow these laws in Haskell if you create a type and make an instance of Applicative for it.

It is still good practice to follow them because:
- you can better reason about what your code is doing
- you can make use of other functions that work with the Applicative type class

## Recap

In this lesson we've discussed:

- the motivation for introducing Applicative type class 

- definition of the Applicative type class

- how to use the `<*>` operator together with `<$>`

- examples that show how to use the Applicative type class

- laws that apply to the Applicative type class